In [90]:
import os
from glob import glob
import pandas as pd
import numpy as np

import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all
import urllib
from urllib.parse import unquote

In [ ]:
df = pd.read_csv("../../data/processed/query_results/topic_lists/indonesia/rec_qids_iwls.csv", sep=',', encoding = 'utf-8') 

## Translation articles - iwlinks match

In [ ]:
#create boolean column: TRUE if 'iwsites' column str contains 'database_code' value
#substring search for the database code in the “iwsites” column, if the database code is set
#[dbcode in iwsites if dbcode is not None else False for (dbcode, iwsites) in zip(translation_rec_iwl_q['database_code'], translation_rec_iwl_q['iwsites'])]
translation_rec_iwl_q['database_code_in_iwsites'] = [x[0] in x[1] if x[0] is not None else False for x in zip(translation_rec_iwl_q['database_code'], translation_rec_iwl_q['iwsites'])]

In [ ]:
translation_rec_iwl_q['database_code_in_iwsites'].values.sum()

In [ ]:
suggestions_created_a = translation_rec_iwl_q[translation_rec_iwl_q['database_code_in_iwsites'] == True]

In [ ]:
count_suggestions_created_a = translation_rec_iwl_q['database_code_in_iwsites'].values.sum()
print(count_suggestions_created_a)

## Translation articles - Qid match

In [ ]:
#objective: count matches by wikidata item in the suggested language

In [ ]:
#use groupby to get a df of articles that DON'T have an interwiki link associated with the 
#suggestion database_code, aka, they weren't created as far as we know so far

located =  translation_rec_iwl_q[translation_rec_iwl_q['database_code_in_iwsites'] == True]
not_yet_located = translation_rec_iwl_q[translation_rec_iwl_q['database_code_in_iwsites'] == False]

In [ ]:
fountain_titles = pd.read_csv("../../data/raw/articles/2019/contest_titles_n_updated.csv", sep=',', encoding = 'utf-8')
fountain_titles_to_cull = fountain_titles[['wiki_db', 'QID']]
fountain_titles_to_cull = fountain_titles_to_cull.rename(columns={'wiki_db': 'database_code'})

In [ ]:
translation_sugg_to_cull = not_yet_located[['database_code','QID']]
suggestions_created_b = pd.merge(fountain_titles_to_cull, not_yet_located, on=['database_code', 'QID'], how='inner')
count_suggestions_created_b = len(suggestions_created_b)
print(count_suggestions_created_b)

## > How many translation articles from the full suggestion list were created since the GLOW contest? 

In [ ]:
def percentage(part, whole):
  return 100 * float(part)/float(whole)

In [ ]:
translation_recs = len(translation_topic_rec_df_CLEAN)
editing_recs = len(editing_topic_rec_df_CLEAN)
edited_count = len(nppt_articles_edits_after_contest_start_unique)
translation_count = len(suggestions_created_a)+len(suggestions_created_b)
print("Total values in full rec list:", len(full_topic_rec_df_ci))
print('***')
print("total recs in translation:", len(translation_topic_rec_df_CLEAN))
print("total recs in editing:", len(editing_topic_rec_df_CLEAN))
print('***')
print(percentage(edited_count,editing_recs),"% created from editing list")
print(percentage(translation_count,translation_recs),"% created from translation list")

In [ ]:
print(len(nppt_articles_edits), "articles from the provided editing list of articles have been edited since the contest started")
print(len(nppt_articles_edits_contest_unique), "of these were edited during the contest")

### > articles were picked from the suggestion lists (out of 34k+)

In [ ]:
print("edited since the contest started", edited_count)
print("edited during the contest", len(nppt_articles_edits_contest_unique))
print("translated since the contest started", translation_count)
print('***')
print("Total since the contest started", edited_count+translation_count)
print("Total during the contest", len(nppt_articles_edits_contest_unique)+translation_count)

In [ ]:
#includes the 1894 with related interwiki links
suggestions_created_a.to_csv("../../data/processed/query_results/topic_lists/suggestions_created_a.csv", sep=',', encoding = 'utf-8', index=False)

In [ ]:
#includes the 61
suggestions_created_b.to_csv("../../data/processed/query_results/topic_lists/suggestions_created_b.csv", sep=',', encoding = 'utf-8', index=False)

In [ ]:
nppt_articles_edits_contest_unique.to_csv("../../data/processed/query_results/topic_lists/stubs_edited_during_contest.csv", sep=',', encoding = 'utf-8', index=False)

#### To Do in the future, topics

TODO:
get latest topic model - see Isaac
Answer the following questions in the future, if time/project level interest:
Q: which topics did editors write about?
Q: what did editors select from these suggestions? 
Q: which topics most resonated or were most popular to write about from these lists? by wiki?
Q: which topics did our partner pass on to us...which search terms made their way to us?